In [68]:
import numpy as np
import pandas as pd
import spacy
from spacy.tokens import Doc, Span, Token
nlp = spacy.load("en_core_web_md")
import re

from itertools import chain

In [69]:
#from spacy import displacy
#import deplacy

In [136]:
# TEST
def get_test_units_list(test_df):
    data = [(row['text'], dict(id=row['essay_id'])) for ind, row in test_df.iterrows()]
    docs = []
    data
    for doc, context in nlp.pipe(data, as_tuples=True):
        doc._.essay_id = context['id']
        docs.append(doc)
        
    segmented_docs = [segmentation(doc, mode='sentence') for doc in docs]
    
    # Flatten lists (Dissolve docs boundaries and store all units together in one huge list)
    units = list(chain.from_iterable(segmented_docs))
    return units

In [151]:
    def get_label(span):
        
        # Gets ADU vs non-ADU LABEL for the span (intended only for sentences)

        # Works if the span is larger or equal to the adu

        # TODO:
        # DOES NOT WORK IF SPAN IS SMALLER THAN ADU, OR IF ADU IS SPLIT BETWEEN TWO SPANS (NEEDS MORE WORK!!!)
        # CLAIM VS PREMISE
        essay_id = span.doc._.essay_id

        span_start = span[0].idx
        span_end = span[-1].idx  + len(span[-1])
        start_inds = adus[adus['essay_id'] == essay_id ]['start_ind'].values
        end_inds = adus[adus['essay_id'] == essay_id ]['end_ind'].values

        # Checks if starting index of span is smaller than ADU and the ending index of the span is larger than the ADU
        return ((start_inds >= span_start) & (end_inds <= span_end)).any()

    
units[0]

Computer has negative effects to children

Nowadays, thanks to the development of technology, computer is now indispensable to life.

In [70]:
doc_features = ['num_tokens', 'para_starts']
span_features = ['word_emb', 'num_tokens', 'num_verbs', 'num_pos_pronouns', 'num_conj_adv', 'num_punct', 'is_para_start',
                 'index_in_doc']
token_features =['word_emb']

# getters that are not used as features
span_utilities = ['prev_unit', 'label']
# methods
span_methods = ['get_nth_unit', 'get_prev_unit_attr']




extensions_dict = dict(doc_features=doc_features, span_features=span_features+span_utilities,
                       token_features=token_features, span_methods=span_methods)





def create_extensions(extensions_dict=None, force=True):
    
    # Features that take 'unit' as input refer to the segmentation, they do not work with just any span.
    
    # Property attributes
    
    # Store starting and ending indices of spans in the whole doc
    # 1 list per each document: [(s1_start, s1_end), (s2_start, s2_end),.., (sn_start, sn_end)]
    Doc.set_extension("units_index_list", default=[],force=True)
    
    # Store essay_id within doc
    Doc.set_extension("essay_id", default=None, force=True)

    
    # Feature Getters
    
    def get_label(span):
        
        # Gets ADU vs non-ADU LABEL for the span (intended only for sentences)

        # Works if the span is larger or equal to the adu

        # TODO:
        # DOES NOT WORK IF SPAN IS SMALLER THAN ADU, OR IF ADU IS SPLIT BETWEEN TWO SPANS (NEEDS MORE WORK!!!)
        # CLAIM VS PREMISE
        essay_id = span.doc._.essay_id

        span_start = span[0].idx
        span_end = span[-1].idx  + len(span[-1])
        start_inds = adus[adus['essay_id'] == essay_id ]['start_ind'].values
        end_inds = adus[adus['essay_id'] == essay_id ]['end_ind'].values

        # Checks if starting index of span is smaller than ADU and the ending index of the span is larger than the ADU
        return ((start_inds >= span_start) & (end_inds <= span_end)).any()
    

    
    
    def get_para_starts(doc):
        # Units starting with \n or preceding \n are considered as paragraph starts
        # if start is 0, start -1 goes back to the last token of the doc

        # TODO
        # para_ends can be obtained by shifing this list to the right by one position
        return [int(doc[start].text =='\n' or doc[start-1].text=='\n') for start, end in doc._.units_index_list]
    
    def get_is_para_start(unit):
        
        para_starts = unit.doc._.para_starts
        unit_ind = unit._.index_in_doc
        
        return para_starts[unit_ind]
        
    
    def get_word_emb(obj):
        return obj.vector
    
    def get_num_tokens(obj):
        return len(obj)
    
    def get_num_verbs(span):
        return sum([1 for token in span if token.pos_ == "VERB"])

    def get_num_pos_pronouns(span):
        return sum([1 for token in span if token.tag_ == "PRP$"])

    def get_num_pron(span):
        return sum([1 for token in span if token.pos_ == "PRON"])
    
    def get_num_conj_adv(span):
        conj_advs = ['moreover', 'incidentally', 'next', 'yet', 'finally', 'then', 'for example', 'thus', 'accordingly', 'namely', 'meanwhile', 'that is', 'also', 'undoubtedly', 'all in all', 'lately', 'hence', 'still', 'therefore', 'in addition', 'indeed', 'again', 'so', 'nevertheless', 'besides', 'instead', 'for instance', 'certainly', 'however', 'anyway', 'further', 'furthermore', 'similarly', 'now', 'in conclusion', 'nonetheless', 'thereafter', 'likewise', 'otherwise', 'consequently']
        return sum([len(re.findall(adv, span.text.lower())) for adv in conj_advs])
    
    def get_num_punct(span):
        return sum([1 for token in span if token.tag_ == "."])
    

    def get_index_in_doc(span):
        """Gets index of the segmented unit in the doc"""
        span_start = span.start

        # span end not used yet
        span_end = span.end

        # finds where span_start is in units_index_list [(s1_start, s1_end), (s2_start, s2_end),.., (sn_start, sn_end)]
        # returns the index of the corresponding span
        return np.where([span.start in range(start, end) for start, end in span.doc._.units_index_list])[0][-1]


    def get_prev_unit(span):

        return span._.get_nth_unit(span._.index_in_doc-1)
    
        
    def get_nth_unit(span, n):

        # Tuple containing the start and end index of the nth span
        span_index = span.doc._.units_index_list[n]

        # Return nth span
        return span.doc[span_index[0]: span_index[1]]



    def get_prev_unit_attr(span, attribute):

        return span._.prev_unit._.get(attribute)
    
    

    # Iterate list of features and Set Extensions (Just to not manually set extensions one by one)
    
    for feature in extensions_dict['doc_features']:
        Doc.set_extension(feature, force=force, getter=locals()[f"get_{feature}"])
        
    for feature in extensions_dict['span_features']:
        Span.set_extension(feature, force=force, getter=locals()[f"get_{feature}"])
        
    for feature in extensions_dict['token_features']:
        Token.set_extension(feature, force=force, getter=locals()[f"get_{feature}"])
        
    for method in extensions_dict['span_methods']:
        Span.set_extension(method, force=force, method=locals()[method])


def segmentation(doc=None ,mode = 'sentence'):
    if mode=='paragraph':
        pass
    if mode=='sentence':
        # segment by sentences
        units = [sent for sent in doc.sents  if not (sent.text.isspace() or sent.text =='')] 
        
        # keep track of (start, end) of units in doc object
        doc._.units_index_list = [(unit.start, unit.end) for unit in units]
        return units
    
    if mode =='avg_n_grams':
        # Code to segment with 15 grams here (average)    
        pass
    if mode=='clause':
        # Code to segment by clause
        pass
    if mode=='token':
        return [token for token in doc if not (token.text.isspace() or token.text =='')]

def unit2fv(unit, feature_list):
    
    fv = np.array([unit._.get(feature) for feature in feature_list], dtype='object')
    
    _fv = np.array([np.reshape(feature, -1) for feature in fv], dtype='object')
    
    return np.concatenate(_fv)

# Run
create_extensions(extensions_dict)   


In [56]:
# Optional, not used yet. Trying to solve problem that title gets included with the first sentence
def add_full_stops(text):
    """adds full stops to texts that end with \n missing full stops"""
    return re.sub("\n+(?!\.)",'.\n', text)
# Not used
def text2doc(text):
    # need to use nlp.pipe here instead
    return nlp(text)

In [57]:
# INPUTS 
essays = pd.read_csv("../data/output_csv/essays.csv")
adus = pd.read_csv("../data/output_csv/adus.csv")

In [58]:
####### TEST
# in_text = essays.iloc[23].text
# doc = nlp(in_text)
# units=segmentation(doc)
# doc._.para_starts

In [91]:
adus

,ADU_index,ADU,essay_id,label,start_ind,end_ind,claim_type,ADU_text
0,T1,MajorClaim 503 575\twe should attach more impo...,essay001,train,503,575,MajorClaim,we should attach more importance to cooperatio...
1,T2,MajorClaim 2154 2231\ta more cooperative attit...,essay001,train,2154,2231,MajorClaim,a more cooperative attitudes towards life is m...
2,T3,"Claim 591 714\tthrough cooperation, children c...",essay001,train,591,714,Claim,"through cooperation, children can learn about ..."
3,T4,Premise 716 851\tWhat we acquired from team wo...,essay001,train,716,851,Premise,What we acquired from team work is not only ho...
4,T5,Premise 853 1086\tDuring the process of cooper...,essay001,train,853,1086,Premise,"During the process of cooperation, children ca..."
...,...,...,...,...,...,...,...,...
6084,T11,Premise 1275 1339\tindirectly they will learn ...,essay402,train,1275,1339,Premise,indirectly they will learn how to socialize ea...
6085,T12,Premise 1341 1388\tThat will make children get...,essay402,train,1341,1388,Premise,That will make children getting lots of friends
6086,T13,Premise 1393 1436\tthey can contribute positiv...,essay402,train,1393,1436,Premise,they can contribute positively to community
6087,T14,Premise 1448 1525\tplaying sport makes childre...,essay402,train,1448,1525,Premise,playing sport makes children getting healthy a...


In [59]:
# Pipelinev1

def text2fv(df):
    data = [(row['text'], dict(id=row['essay_id'])) for ind, row in df.iterrows()]
    docs = []
    data
    for doc, context in nlp.pipe(data, as_tuples=True):
        doc._.essay_id = context['id']
        docs.append(doc)
        
    segmented_docs = [segmentation(doc, mode='sentence') for doc in docs]
    
    # Flatten lists (Dissolve docs boundaries and store all units together in one huge list)
    units = list(chain.from_iterable(segmented_docs))
    
    X_features = span_features
    

    X = np.array([unit2fv(unit, X_features) for unit in units])
    y = np.array([int(unit._.label) for unit in units])
    
    return X,y 

In [60]:
train = essays[essays['label'] =='train']
test =essays[essays['label'] =='test']

X_train, y_train = text2fv(train)

X_test, y_test = text2fv(test)



In [197]:
# TEST


essay24adus = adus[adus['essay_id'] == 'essay024']


In [174]:
ind_list = doc._.units_index_list

In [175]:
ind_list

[(0, 23),
 (23, 46),
 (46, 56),
 (56, 70),
 (70, 98),
 (98, 124),
 (124, 149),
 (149, 186),
 (186, 209),
 (209, 238),
 (238, 269),
 (269, 270),
 (270, 298),
 (298, 329),
 (329, 330)]

In [186]:
doc[22]

.

In [214]:
start_unit = [doc[start].idx for start, end in ind_list]
end_unit = [ doc[end-1].idx+1 for start, end in ind_list]


In [228]:
start_end = essay24adus[['start_ind', 'end_ind']]

for i, row in start_end.iterrows():
    span_s = np.where(np.array(start_unit) <row['start_ind'])[0][-1]
    span_e = np.where(np.array(end_unit) <row['end_ind'])[0][-1]
    
    print(row['start_ind'], )

12

In [210]:
np.where(np.array(start_unit) <1480)[0][-1]

[(0, 132),
 (133, 260),
 (261, 303),
 (303, 374),
 (375, 522),
 (522, 664),
 (665, 796),
 (797, 974),
 (974, 1094),
 (1095, 1240),
 (1241, 1391),
 (1391, 1392),
 (1392, 1531),
 (1532, 1704),
 (1704, 1705)]

In [222]:
np.where(np.array(start_unit) <1480)[0][-1]

12

In [223]:
start_unit[12]

1392

In [216]:
end_unit

[132,
 260,
 303,
 374,
 522,
 664,
 796,
 974,
 1094,
 1240,
 1391,
 1392,
 1531,
 1704,
 1705]

# Classification

In [78]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

In [79]:
logreg = LogisticRegression(solver='newton-cg')
logreg.fit(X_train, y_train)


LogisticRegression(solver='newton-cg')

In [80]:
preds_lr = logreg.predict(X_test)
print(classification_report(y_test, preds_lr))

              precision    recall  f1-score   support

           0       0.78      0.45      0.57       250
           1       0.89      0.97      0.93      1134

    accuracy                           0.88      1384
   macro avg       0.83      0.71      0.75      1384
weighted avg       0.87      0.88      0.86      1384



In [83]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [84]:
rf.fit(X_train, y_train)

RandomForestClassifier()

array([[-4.33700159e-03,  5.74849825e-03, -8.07924047e-02, ...,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.06555268e-01,  2.89857566e-01, -5.34156524e-02, ...,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [-1.15756266e-01,  2.25932971e-01, -5.61617054e-02, ...,
         1.00000000e+00,  0.00000000e+00,  2.00000000e+00],
       ...,
       [-4.53010015e-02,  2.28496000e-01, -2.38107115e-01, ...,
         1.00000000e+00,  0.00000000e+00,  1.40000000e+01],
       [-7.72796273e-02,  2.66258746e-01, -1.75650015e-01, ...,
         1.00000000e+00,  0.00000000e+00,  1.50000000e+01],
       [-2.38838326e-02,  2.44683713e-01, -1.54835701e-01, ...,
         1.00000000e+00,  1.00000000e+00,  1.60000000e+01]])

In [85]:
preds = rf.predict(X_test)

confusion_matrix(y_test, preds)

array([[ 145,  105],
       [  19, 1115]], dtype=int64)

In [86]:
preds

array([0, 1, 1, ..., 1, 1, 1])

In [101]:
X_test

array([[-2.30584666e-02,  1.70649409e-01,  1.19616408e-02, ...,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 4.34298851e-02,  9.00983885e-02, -8.72147977e-02, ...,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 1.32289603e-01,  7.90781230e-02, -1.53517351e-01, ...,
         1.00000000e+00,  1.00000000e+00,  2.00000000e+00],
       ...,
       [-5.44680282e-02,  8.68857130e-02, -1.39478177e-01, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+01],
       [-2.51547489e-02,  1.78158998e-01, -4.51728217e-02, ...,
         1.00000000e+00,  0.00000000e+00,  1.10000000e+01],
       [-2.11575646e-02,  1.66456774e-01, -1.57130525e-01, ...,
         1.00000000e+00,  1.00000000e+00,  1.20000000e+01]])

In [32]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.88      0.64      0.74       262
           1       0.92      0.98      0.95      1135

    accuracy                           0.92      1397
   macro avg       0.90      0.81      0.85      1397
weighted avg       0.92      0.92      0.91      1397



## CrossValidation

In [26]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import SGDClassifier
from sklearn.base import clone

In [27]:
""" Stochastic Gradient Descent (SGD) classifier, 
This classifier has the advantage of being capable of handling very large datasets efficiently"""
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train)

SGDClassifier(random_state=42)

In [47]:
skfolds = StratifiedKFold(n_splits=5, random_state=42)

best_model = None 
precision = 0
for train_index, test_index in skfolds.split(X_train, y_train):
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train[train_index]
    y_train_folds = y_train[train_index]
    X_test_fold = X_train[test_index]
    y_test_fold = y_train[test_index]
    
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    if precision < n_correct / len(y_pred):
        best_model = clone_clf
        precision = n_correct / len(y_pred)
    print(n_correct / len(y_pred))
    print(confusion_matrix(y_test_fold, y_pred))
    

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


0.7224231464737794
[[181  53]
 [254 618]]
0.8707052441229657
[[145  89]
 [ 54 818]]
0.8471971066907775
[[110 124]
 [ 45 827]]
0.8426763110307414
[[ 69 164]
 [ 10 863]]
0.8090497737556561
[[183  50]
 [161 711]]


In [48]:
preds = best_model.predict(X_test)

confusion_matrix(y_test, preds)

array([[ 173,   89],
       [  81, 1054]])

In [50]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.68      0.66      0.67       262
           1       0.92      0.93      0.93      1135

    accuracy                           0.88      1397
   macro avg       0.80      0.79      0.80      1397
weighted avg       0.88      0.88      0.88      1397



In [78]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [91]:
svm_clf = LinearSVC(random_state=0, tol=1e-5, verbose=1, max_iter=50000)

In [92]:
svm_clf.fit(X_train, y_train)

[LibLinear]

/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC(max_iter=50000, random_state=0, tol=1e-05, verbose=1)

In [95]:
preds = svm_clf.predict(X_test)

confusion_matrix(y_test, preds)

array([[ 127,  135],
       [  35, 1100]])

In [96]:
print(classification_report(y_test, preds))


              precision    recall  f1-score   support

           0       0.78      0.48      0.60       262
           1       0.89      0.97      0.93      1135

    accuracy                           0.88      1397
   macro avg       0.84      0.73      0.76      1397
weighted avg       0.87      0.88      0.87      1397



In [97]:
from sklearn import svm
svm_clf = svm.SVC(kernel='linear')


In [98]:
svm_clf.fit(X_train, y_train)

SVC(kernel='linear')

In [99]:
preds = svm_clf.predict(X_test)

confusion_matrix(y_test, preds)

array([[ 132,  130],
       [  32, 1103]])

In [100]:
print(classification_report(y_test, preds))


              precision    recall  f1-score   support

           0       0.80      0.50      0.62       262
           1       0.89      0.97      0.93      1135

    accuracy                           0.88      1397
   macro avg       0.85      0.74      0.78      1397
weighted avg       0.88      0.88      0.87      1397



### Hard Voting 

In [107]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [105]:
log_clf = LogisticRegression(solver='newton-cg')
rnd_clf = RandomForestClassifier()
smv_clf = SVC()

In [106]:
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', smv_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(solver='newton-cg')),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [108]:
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8869005010737294
RandomForestClassifier 0.9112383679312813
SVC 0.8840372226198998
VotingClassifier 0.9226914817465999


In [114]:
preds = voting_clf.predict(X_test)

confusion_matrix(y_test, preds)

array([[ 176,   86],
       [  22, 1113]])

In [115]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.89      0.67      0.77       262
           1       0.93      0.98      0.95      1135

    accuracy                           0.92      1397
   macro avg       0.91      0.83      0.86      1397
weighted avg       0.92      0.92      0.92      1397



## Bagging and Pasting

In [109]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [110]:
bag_clf = BaggingClassifier(
        DecisionTreeClassifier(), n_estimators=500,
        max_samples=100, bootstrap=True, n_jobs=-1)

In [111]:
bag_clf.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=-1)

In [112]:
preds = bag_clf.predict(X_test)

confusion_matrix(y_test, preds)

array([[ 179,   83],
       [  61, 1074]])

              precision    recall  f1-score   support

           0       0.75      0.68      0.71       262
           1       0.93      0.95      0.94      1135

    accuracy                           0.90      1397
   macro avg       0.84      0.81      0.83      1397
weighted avg       0.89      0.90      0.90      1397

